# MLflow

>[MLflow](https://mlflow.org/) is a versatile, open-source platform for managing workflows and artifacts across the machine learning and generative AI lifecycle. It has built-in integrations with many popular AI and ML libraries, but can be used with any library, algorithm, or deployment tool. It is designed to be extensible, so you can write plugins to support new workflows, libraries, and tools.

MLflow's LangChain integration provides the following capabilities:

- **Tracing**: MLflow allows you to visually trace data flows through your LangChain chains, agents, retrievers, or other components, all with just one line of code (`mlflow.langchain.autolog()`).
- **Experiment Tracking**: MLflow tracks and stores artifacts from your LangChain experiments, including models, code, prompts, metrics, and more.
- **Dependency Management**: MLflow automatically records model dependencies, ensuring consistency between development and production environments.
- **Model Evaluation** MLflow offers native capabilities for evaluating LangChain applications.

**Note**: MLflow tracing is available in MLflow versions 2.14.0 and later.

This notebook demonstrates how to track your LangChain experiments using MLflow. For more information about this feature and to explore tutorials and examples of using LangChain with MLflow, please refer to the [MLflow documentation for LangChain integration](https://mlflow.org/docs/latest/llms/langchain/index.html). If you're new to MLflow, check out the [Getting Started with MLflow](https://mlflow.org/docs/latest/getting-started/index.html) guide.

## Setup

Install MLflow Python package:

In [ ]:
%pip install mlflow -qU

This example will use OpenAI's GPT-4o model. Feel free to skip the command below and proceed with a different LLM if desired.

In [ ]:
%pip install langchain-openai -qU

In [4]:
import os

# Set MLflow tracking URI if you have MLflow Tracking Server running
os.environ["MLFLOW_TRACKING_URI"] = ""
os.environ["OPENAI_API_KEY"] = ""

To begin, let's create a dedicated [MLflow experiment](https://mlflow.org/docs/latest/tracking.html#experiments) in order track our model and artifacts. While you can opt to skip this step and use the default experiment, we strongly recommend organizing your runs and artifacts into separate experiments to avoid clutter and maintain a clean, structured workflow.

In [ ]:
import mlflow

mlflow.set_experiment("LangChain MLflow Integration")

## Overview

Integrate MLflow with your LangChain Application using one of the following methods:

1. **Autologging**: Enable comprehensive tracing with the `mlflow.langchain.autolog()` command. MLflow tracing will record your LangChain chains, agents, retrievers, or other components, all with just one line of code, make it much easier to understand your chain's behavior and to identify bugs.
2. **Manual Logging**: Use MLflow APIs to log LangChain chains and agents, enabling you to record and version your chains and agents, providing fine-grained control over what to track in your experiment.
3. **Custom Callbacks**: Pass MLflow callbacks manually when invoking chains, allowing for semi-automated customization of your workload, such as tracking specific invocations.

## Scenario 1: MLFlow Autologging

To get started with MLflow tracing for LangChain, simply call `mlflow.langchain.autolog()`. After running this one line of code, MLflow will record all calls to your LangChain models as [traces](https://mlflow.org/docs/latest/llms/tracing/index.html), which you can view in the MLflow UI.

In [3]:
import mlflow

mlflow.langchain.autolog()

### Define a Chain

Now, we will define a chain, run it, and examine the MLflow UI to see the trace.

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)
parser = StrOutputParser()

chain = prompt | llm | parser

### Invoke the Chain

In [ ]:
test_input = {
    "input_language": "English",
    "output_language": "German",
    "input": "I love programming.",
}

chain.invoke(test_input)

Now let's inspect the traces in the MLflow UI. You can launch the UI by running `mlflow ui` in your terminal. You can find the traces by navigating to the experiment you created above (`LangChain MLflow Integration`) and clicking on the "Traces" tab.

![MLflow Trace](/img/mlflow_1_trace.png)

The displayed trace visualizes the call stack of your chain invocation, providing you with a deep insight into how each component is executed within the chain.


### Tracing LangGraph

LangGraph graph invocations are also traced automatically when you enable autologging with `mlflow.langchain.autolog()`. You can install LangGraph with `pip install langgraph`

In [ ]:
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent


@tool
def count_words(text: str) -> str:
    """Counts the number of words in a text."""
    word_count = len(text.split())
    return f"This text contains {word_count} words."


llm = ChatOpenAI(model="gpt-4o")
tools = [count_words]
graph = create_react_agent(llm, tools)

result = graph.invoke(
    {"messages": [{"role": "user", "content": "Write me a 71-word story about a cat."}]}
)

We can, again, find this trace in the MLflow UI.

![LangGraph Trace](/img/mlflow_2_langgraph_trace.png)

### More about Autologging

By default, `mlflow.langchain.autolog()` only records traces; it does not log any models. You can configure it to log models by setting the `log_models` parameter to `True`. When working with LangChain and LangGraph, however, manual model logging using [MLflow's models-from-code logging](https://mlflow.org/docs/latest/llms/langchain/index.html#logging-models-from-code) approach is recommended. We will cover this in the next section.

For a comprehensive list of available configurations, please refer to the latest [MLflow LangChain Autologging Documentation](https://mlflow.org/docs/latest/llms/langchain/autologging.html).

## Scenario 2: Manually Logging an Agent from Code

### Define an Agent

In this example, we'll log our tip calculator agent to MLflow. Model logging provides several key benefits:

1. **Version Control**: Track different versions of your agents and chains as you experiment and improve them
2. **Reproducibility**: Ensure your agent can be recreated exactly as it was when you logged it
3. **Deployment Ready**: Logged models can be easily loaded and deployed in other environments
4. **Documentation**: The logged code serves as clear documentation of how your agent works

This example uses MLflow's [models-from-code](https://mlflow.org/docs/latest/llms/langchain/index.html#logging-models-from-code) approach to model logging, which stores the model definition as Python code rather than using serialization. This approach is particularly valuable for LangChain applications since many components (like lambda functions or external connections) cannot be properly serialized.

To log your LangChain model, create a separate `.py` file that defines the agent instance.

In the interest of time, you can run the following cell to generate a Python file `agent.py`, which contains the agent definition code, using the `%%writefile` magic command. In actual dev scenario, you would define it in another notebook or hand-crafted python script.

In [52]:
%%writefile agent.py
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
import mlflow

@tool
def calculate_tip(bill_amount: float, tip_percent: float) -> str:
    """Calculate tip amount and total bill.
    Args:
        bill_amount: The total amount of the bill in dollars
        tip_percent: The tip percentage (e.g., 20 for 20%)
    """
    tip_amount = bill_amount * (tip_percent / 100)
    total = bill_amount + tip_amount
    return f"For a bill of ${bill_amount:.2f} with {tip_percent}% tip:\\nTip amount: ${tip_amount:.2f}\\nTotal bill: ${total:.2f}"

llm = ChatOpenAI(model_name="gpt-4o")
tools = [calculate_tip]

agent = create_react_agent(llm, tools)

# IMPORTANT: call set_model() to register the instance to be logged.
mlflow.models.set_model(agent)


### Log the Agent

Return to the original notebook and run the following cell to log the agent you've defined in the `agent.py` file.


In [ ]:
messages = {
    "messages": [
        {
            "role": "user",
            "content": "What is the tip for a $216.32 bill with a 22% tip?",
        }
    ]
}

with mlflow.start_run(run_name="tip-agent") as run:
    info = mlflow.langchain.log_model(
        lc_model="agent.py",  # Specify the relative code path to the agent definition
        artifact_path="model",
        input_example=messages,
    )

print("The agent is successfully logged to MLflow!")

Now, open the MLflow UI and navigate to the "Artifacts" tab in the Run detail page. You should see that the `agent.py` file has been successfully logged, along with other model artifacts, such as dependencies, input examples, and more.

![MLflow Artifacts](/img/mlflow_3_model_from_code.png)

### Invoke the Logged Agent

Now load the agent back and invoke it. There are two ways to load the model

In [ ]:
# Let's turn on the autologging with default configuration, so we can see the trace for the agent invocation.
mlflow.langchain.autolog()

# Load the model back
agent = mlflow.pyfunc.load_model(info.model_uri)

# Invoke
agent.predict(messages)

As before, we can inspect the trace in the MLflow UI.

![Logged Model Trace](/img/mlflow_4_trace_logged.png)

## Scenario 3. Using MLflow Callbacks

MLflow provides a callback-based approach to trace your LangChain applications. The `MlflowLangchainTracer` callback allows you to generate traces for specific chain or agent invocations, providing more granular control over what gets traced compared to autologging.

### MlflowLangchainTracer

When the chain or agent is invoked with the `MlflowLangchainTracer` callback, MLflow will automatically generate a trace for the call stack and log it to the MLflow tracking server.  The outcome is exactly same as `mlflow.langchain.autolog()`, but this is particularly useful when you want to only trace specific invocation. Autologging is applied to all invocation in the same notebook/script, on the other hand.

In [ ]:
from mlflow.langchain.langchain_tracer import MlflowLangchainTracer

mlflow_tracer = MlflowLangchainTracer()

# This call generates a trace
chain.invoke(test_input, config={"callbacks": [mlflow_tracer]})

# This call does not generate a trace
chain.invoke(test_input)

#### Where to Pass the Callback
LangChain supports two ways of passing callback instances: (1) Request time callbacks: pass them to the `invoke` method or bind with `with_config()` (2) Constructor callbacks: set them in the chain constructor. When using the `MlflowLangchainTracer` as a callback, you **must use request time callbacks**. Setting it in the constructor instead will only apply the callback to the top-level object, preventing it from being propagated to child components, resulting in incomplete traces. For more information on this behavior, please refer to [Callbacks Documentation](https://python.langchain.com/docs/concepts/callbacks) for more details.

```python
# Recommended: these methods will work correctly
chain.invoke(test_input, config={"callbacks": [mlflow_tracer]})
chain.with_config(callbacks=[mlflow_tracer])
# Not recommended: this will result in incomplete traces
chain = TheNameOfSomeChain(callbacks=[mlflow_tracer])
```

#### Supported Methods

`MlflowLangchainTracer` supports the following invocation methods from the [Runnable Interfaces](https://python.langchain.com/v0.1/docs/expression_language/interface/).
-  Standard interfaces: `invoke`, `stream`, `batch`
-  Async interfaces: `astream`, `ainvoke`, `abatch`, `astream_log`, `astream_events`

Other methods are not guaranteed to be fully compatible.

## Conclusion

In this guide, we have seen three main ways to track your LangChain applications using MLflow:

1. **Autologging**: Enable comprehensive tracing with the `mlflow.langchain.autolog()` command. MLflow tracing will record your LangChain chains, agents, retrievers, or other components, all with just one line of code, make it much easier to understand your chain's behavior and to identify bugs.
2. **Manual Logging**: Use MLflow APIs to log LangChain and LangGraph models, enabling you to record and version your chains and agents.
3. **Custom Callbacks**: Use MLflow callbacks to trace specific chain or agent invocations when you want more granular control. Instead of tracing everything with autologging, you can add the MLflow tracer only to the specific calls you want to monitor.


## References
To learn more about the feature and visit tutorials and examples of using LangChain with MLflow, please refer to the [MLflow documentation for LangChain integration](https://mlflow.org/docs/latest/llms/langchain/index.html).

`MLflow` also provides several [tutorials](https://mlflow.org/docs/latest/llms/langchain/index.html#getting-started-with-the-mlflow-langchain-flavor-tutorials-and-guides) and [examples](https://github.com/mlflow/mlflow/tree/master/examples/langchain) for the `LangChain` integration:
- [Quick Start](https://mlflow.org/docs/latest/llms/langchain/notebooks/langchain-quickstart.html)
- [RAG Tutorial](https://mlflow.org/docs/latest/llms/langchain/notebooks/langchain-retriever.html)
- [Agent Example](https://github.com/mlflow/mlflow/blob/master/examples/langchain/simple_agent.py)